In [7]:
from datetime import datetime, timedelta
from pathlib import Path

from adcircpy import AdcircMesh, AdcircRun, Tides
from adcircpy.forcing.winds import BestTrackForcing
from adcircpy.utilities import download_mesh

import numpy as np

stage = 'staging'
HURRICANE = 'Matthew2016'
OUTPUT_DIRECTORY = stage+'/runs/'+HURRICANE+'_test2'

In [8]:
# open mesh file
mesh = AdcircMesh.open('datasets/nc_inundation_v6c.grd', crs=4326)    

# Adding Manning's
# generate tau0 factor
mesh.generate_tau0()

# also add Manning's N to the domain (constant for this example)
mesh.mannings_n_at_sea_floor = np.full(mesh.values.shape, 0.025)

# initialize tidal forcing and constituents
tidal_forcing = Tides()
tidal_forcing.use_all()
mesh.add_forcing(tidal_forcing)

/home/alexandre-georges/anaconda3/envs/adcirc/lib/python3.12/site-packages/adcircpy/mesh/parsers/grd.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  output['nodes'] = pandas.read_csv(
/home/alexandre-georges/anaconda3/envs/adcirc/lib/python3.12/site-packages/adcircpy/mesh/parsers/grd.py:37: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  output['elements'] = pandas.read_csv(


In [9]:
# set simulation dates
spinup_time = timedelta(days=15)                                                                        
duration = timedelta(days=11)
start_date = datetime(2016, 9, 28, 12)
end_date = start_date + duration

In [10]:
# initialize wind forcing
wind_forcing = BestTrackForcing(storm=HURRICANE, nws=20)
mesh.add_forcing(wind_forcing)

/home/alexandre-georges/anaconda3/envs/adcirc/lib/python3.12/site-packages/stormevents/nhc/track.py:1137: RuntimeWarning: invalid value encountered in divide
  speeds = pandas.Series(distances / abs(intervals), index=indices)


In [11]:
# instantiate driver object
driver = AdcircRun(mesh, start_date, end_date, spinup_time, netcdf=True)   
driver.NOUTGE = -3
driver.NOUTGV = -3
driver.NOUTGM = -3
driver.TOUTFGE = 30
driver.TOUTFGV = 30
driver.TOUTFGM = 30
driver.NSPOOLGE = 20
driver.NSPOOLGV = 20
driver.NSPOOLGM = 20
                                                                                                                                                  # write driver state to disk
driver.write(OUTPUT_DIRECTORY, overwrite=True)

In [12]:
bash_script = rf"""
#!/bin/bash

current_dir=$(pwd)
subdomain_dir=$current_dir/s1_gonave
folder_name=$(basename $current_dir)

mkdir ../../logs
mkdir ../../logs/$folder_name
log_header="Logs of run attempts and their parameters."
log_header2="Hurricane $folder_name"
log_header3="------------------------------------------"
log_header4="id\t\t\t\t\t\tdt\t\t\t\t\t\ttype\t\t\t\t\t\tcomment"
echo -e $log_header > runlog.out
echo -e $log_header2 >> runlog.out
echo -e $log_header3 >> runlog.out
echo -e $log_header4 >> runlog.out

cp ../../../scripts/fort22fix.py fort22fix.py
python fort22fix.py

cp ../../../scripts/run_new.sh run.sh
cp ../../../scripts/slurmcold.sh slurm.sh

current_dir=$(pwd)
folder_name=$(basename $current_dir)
sed -i "s/^#SBATCH --job-name=.*$/#SBATCH --job-name=$folder_name/" slurm.sh

# subdomain file structure setup
mkdir s1_gonave
subdomain_dir=$current_dir/s1_gonave
cp ../../../datasets/s1_gonave/shape.e14 s1_gonave/shape.e14
# remove BOM (\ufeff) from shape.e14
sed -i '1s/^\xef\xbb\xbf//' s1_gonave/shape.e14

source ~/anaconda3/etc/profile.d/conda.sh  # Adjust the path to your Conda installation
conda activate py2
python2.7 ../../../scripts/gensub.py $current_dir $subdomain_dir

chmod +x run.sh slurm.sh 

"""

# Write the bash script to the output directory
with open(OUTPUT_DIRECTORY + '/init.sh', 'w') as file:
    file.write(bash_script)

print("Bash script created and outputted at", OUTPUT_DIRECTORY)

Bash script created and outputted at staging/runs/Matthew2016_test2
